
# Data Preparation
### Preprocessing the data needed from the dataset


In [1]:
import numpy as np
import tensorflow as tf
import tflearn
import random
from tensorflow.python.framework import ops

#Usde to for Contextualisation and Other NLP Tasks.
import nltk
from nltk.stem.lancaster import LancasterStemmer
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
stemmer = LancasterStemmer()

#Other
import json
import pickle
import warnings
warnings.filterwarnings("ignore")

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tahjy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tahjy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load json and extract required data
with open('intents.json') as json_data:
    intents = json.load(json_data)

# Looping through the Intents to Convert them to words, classes, documents and ignore_words.......
words = []
classes = []
documents = []
ignore_words = set(stopwords.words('english'))
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])


In [3]:
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

45 documents
6 classes ['about', 'consultation', 'goodbye', 'greeting', 'name', 'thanks']
54 unique stemmed words ['!', '?', 'afternoon', 'anyon', 'big', 'bless', 'body', 'bonjo', 'bye', 'cal', 'can', 'could', 'day', 'ev', 'feel', 'giv', 'going', 'good', 'goodby', 'greet', 'gwan', 'hand', 'hay', 'hello', 'help', 'hey', 'hi', 'hol', 'howdy', 'i', 'is', 'lat', 'lik', 'morn', 'nam', 'nee', 'new', 'okay', 'pleas', 'real', 'see', 'sick', 'sup', 'support', 'thank', 'that', 'up', 'uptop', 'wah', 'wel', 'what', 'whatâ€™s', 'who', 'yo']


In [1]:

#Creating the Data for our Model
training = []
output = []

output_empty = [0] * len(classes)

#Creating Traning Set, Bag of Words for our Model

for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
    print(bag)


NameError: name 'classes' is not defined

In [5]:
#Shuffling Randomly and Converting into Numpy Array for Faster Processing
random.shuffle(training)
training = np.array(training)

#Creating Train and Test Lists
train_x = list(training[:,0])
train_y = list(training[:,1])

#Building Neural Network for Out Chatbot to be Contextual
ops.reset_default_graph()

# Model Training

In [6]:
# define our Neural Network architecture using tflearn

net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [7]:

#Training the model
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)

#Saving the model
model.save('model.tflearn')


Training Step: 5999  | total loss: 0.00499 | time: 0.015s
| Adam | epoch: 1000 | loss: 0.00499 - acc: 1.0000 -- iter: 40/45
Training Step: 6000  | total loss: 0.00479 | time: 0.018s
| Adam | epoch: 1000 | loss: 0.00479 - acc: 1.0000 -- iter: 45/45
--
INFO:tensorflow:C:\Users\tahjy\Documents\Web Apps\lepios_project_backend\lepios_project_backend\chat_model\assets\chatbot\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [8]:
# Save our training data which consists of the classes/tags, corpus
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

# Implementing Chat Function

In [9]:

# Load our pickled training dataset
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']


with open('intents.json') as json_data:
    intents = json.load(json_data)
    
# load our saved model
model.load('./model.tflearn')



INFO:tensorflow:Restoring parameters from C:\Users\tahjy\Documents\Web Apps\lepios_project_backend\lepios_project_backend\chat_model\assets\chatbot\model.tflearn


In [13]:
def clean_up_sentence(sentence):
    # It Tokenize or Break it into the constituents parts of Sentense.
    sentence_words = nltk.word_tokenize(sentence)
    # Stemming means to find the root of the word.
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# Return the Array of Bag of Words: True or False and 0 or 1 for each word of bag that exists in the Sentence
def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

# Define our error threshold
ERROR_THRESHOLD = 0.20

def classify(sentence):
    # Prediction or To Get the Posibility or Probability from the Model
    results = model.predict([bow(sentence, words)])[0]
    # Exclude those results which are Below Threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # Sorting is Done because heigher Confidence Answer comes first.
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1])) #Tuppl -> Intent and Probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # That Means if Classification is Done then Find the Matching Tag.
    if results:
        print(results)
        # Long Loop to get the Result.
        while results:
            for i in intents['intents']:
                # Tag Finding
                if i['tag'] == results[0][0]:
                    # Random Response from High Order Probabilities
                    return print(random.choice(i['responses']))

            results.pop(0)

In [ ]:
print("Start messaging with the bot")

while True:
    input_data = input("You- ")
    answer = response(input_data)
    answer



Start messaging with the bot
You- Sup
[('greeting', 0.9969599)]
Hi there, need a health checkup?
